In [2]:
import tensorflow.keras as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
import tf2onnx
import numpy as np
import math
from pathlib import Path
from keras.utils.np_utils import to_categorical

In [6]:
batch_size = 500 # Number of images processed at once
nb_classes = 10  # 10 Digits from 0 to 9

# Dimensionen of the input images (28x28 pixel)
img_rows, img_cols = 28, 28

# Load image data with labels, split into test and training set 
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# reshape images in 4D tensor (N images, 28 rows, 28 columns, 1 channel) 
# rescale pixels range from [0, 255] to [0, 1]
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
print('X_train shape: ', X_train.shape)
print(X_train.shape[0], "training samples")
print(X_test.shape[0], "test samples")

# convert digit labels (0-9) in one-hot encoded binary vectors. 
# These correspond to the training/test labels at the output of the net. 
Y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
Y_test = tf.keras.utils.to_categorical(y_test, nb_classes)
print("One-hot encoding: {}".format(Y_train[0, :]))

X_train shape:  (60000, 28, 28, 1)
60000 training samples
10000 test samples
One-hot encoding: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() ## for Lenet5

In [8]:
class DataGenerator(tf.compat.v2.keras.utils.Sequence):
 
    def __init__(self, X_data , y_data, batch_size, dim, n_classes,
                 to_fit, shuffle = True):
        self.batch_size = batch_size
        self.X_data = X_data
        self.labels = y_data
        self.y_data = y_data
        self.to_fit = to_fit
        self.n_classes = n_classes
        self.dim = dim
        self.shuffle = shuffle
        self.n = 0
        self.list_IDs = np.arange(len(self.X_data))
        self.on_epoch_end()
    def __next__(self):
        # Get one batch of data
        data = self.__getitem__(self.n)
        # Batch index
        self.n += 1
        
        # If we have processed the entire dataset then
        if self.n >= self.__len__():
            self.on_epoch_end
            self.n = 0
        
        return data
    def __len__(self):
        # Return the number of batches of the dataset
        return math.ceil(len(self.indexes)/self.batch_size)
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:
            (index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X = self._generate_x(list_IDs_temp)
        
        if self.to_fit:
            y = self._generate_y(list_IDs_temp)
            return X, y
        else:
            return X
    def on_epoch_end(self):
        
        self.indexes = np.arange(len(self.X_data))
        
        if self.shuffle: 
            np.random.shuffle(self.indexes)
    def _generate_x(self, list_IDs_temp):
               
        X = np.empty((self.batch_size, *self.dim))
        
        for i, ID in enumerate(list_IDs_temp):
            
            X[i,] = self.X_data[ID]
            
            # Normalize data
            X = (X/255).astype('float32')
            
        return X[:,:,:, np.newaxis]
    def _generate_y(self, list_IDs_temp):
        
        y = np.empty(self.batch_size)
        
        for i, ID in enumerate(list_IDs_temp):
            
            y[i] = self.y_data[ID]
            
        return keras.utils.to_categorical(y,num_classes=self.n_classes)

In [9]:
n_classes = 10
nb_classes = 10
input_shape = (28, 28) #Lenet5

In [10]:
training_id = 4
model_short_name = 'Lenet5'
framework = 'keras'

In [11]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/Keras/lenet5/'
since_0 = time.time()
#model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
#model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
model_name = 'tf_Lenet5_mnist_2021-10-27_{}'.format(training_id)
model = tf.keras.models.load_model(path+ model_name+'.h5')
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+ model_name+'.h5')
size0

601864

In [12]:
import onnx
import onnxruntime
import coremltools
import time

In [13]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/miss-classification/v2/'

In [14]:
from sklearn import metrics
def model_scores(y_test, test_predict):
    total_ = np.sum(y_test == y_test)
    correct_ = np.sum(y_test == test_predict)
    accuracy2  = correct_ #*100./total_
    return accuracy2 #, precision2, recall2, f12
def get_miss_classification(y1, y2):
    miss_perc_val_original_coreml = 0
    try:
        np.testing.assert_array_equal(y1[0], y2[0])
    except Exception as e:
        miss_perc_val_original_coreml = 1
    return miss_perc_val_original_coreml #, precision2, recall2, f12

In [15]:
def to_onnx(i, x, y, data_writer_run, batch_size):
    
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
    #print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    
    ### Original Model
    since_1 = time.time()
    #model = torch.load(path+model_name+'.pth')
    with tf.device('/cpu:0'): 
        k_predict = model.predict(x)
    inference_time_original = time.time() - since_1
    y0 = to_categorical(np.argmax(k_predict, 1), num_classes = 10)
    correct_original = np.sum(y0 == y)
    accuracy_original = model_scores(y, y0)
    miss_test_original = get_miss_classification(y, y0)
    # ONNX Model
    
    t_elapsed_2 = time.time() - since_1
    #since_1 = time.time()
    #onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #load_time_onnx = time.time() - since_1
    #onnx.checker.check_model(onnx_model)
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    #ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: x}
    ort_outs = ort_session.run(None, ort_inputs)
    inference_time_onnx = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    #print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    ####### Mis-classification ONNX ######################################
    y2 = to_categorical(np.argmax(ort_outs[0], 1), num_classes = 10)
    correct_onnx = np.sum(y2 == y)
    accuracy_onnx = model_scores(y, y2)
    
    miss_original_onnx = get_miss_classification(y0, y2)
    miss_test_onnx = get_miss_classification(y, y2)
    ####### End of mis-classification ONNX ###################################### 
    
    
    ## CoreML
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    #spec = coreml_model.get_spec()
    #coreml_model = coremltools.models.MLModel(spec)
    
    since_1 = time.time()
    output_dict_test = coreml_model.predict({'conv2d_input':x})
    inference_time_coreml = time.time() - since_1
    ####### Mis-classification coreML ######################################
    y3 = to_categorical(np.argmax(output_dict_test['Identity'], 1), num_classes = 10)
    correct_coreml = np.sum(y3 == y)
    accuracy_coreml = model_scores(y, y3)
    
    #print(correct_original, correct_coreml, correct_onnx, np.sum(y == y))
    ## Part 1
    ####### Part 2
    #print('default-shape: ',k_predict.shape, 'onnx-shape: ',ort_outs[0].shape, 'coreml-shape: ',output_dict_test['Identity'].shape)
    miss_original_coreml = get_miss_classification(y0, y3)
    miss_test_coreml = get_miss_classification(y, y3)
    ####### End of mis-classification coreML ######################################
    #return correct_original,correct_onnx,correct_coreml
    list_val = [accuracy_original, accuracy_onnx, accuracy_coreml, miss_test_original, miss_original_onnx, miss_test_onnx, miss_original_coreml, miss_test_coreml,'', inference_time_original, inference_time_onnx, inference_time_coreml]
    return list_val

In [16]:
def _lets_convert(data, data_writer_run, batch_size):
    since = time.time()
    accuracy_original = 0.0
    accuracy_onnx = 0.0
    accuracy_coreml = 0.0
    
    miss_test_original = 0.0
    miss_original_onnx = 0
    miss_test_onnx = 0
    miss_original_coreml = 0
    miss_test_coreml = 0
    
    inference_time_original = 0.0
    inference_time_onnx = 0.0
    inference_time_coreml = 0.0
    
    total_ = 0.0
    total_datasets = 0
    for i, (images, labels) in enumerate(data):
        #torch.cuda.empty_cache()
        #images = images.cuda()
        #for j in range(len(images)):
        list_val = to_onnx(i, images,labels, data_writer_run, batch_size)
        
        if i%50 == 0:
            print(i, list_val)
        accuracy_original += list_val[0]
        accuracy_onnx += list_val[1]
        accuracy_coreml += list_val[2]
        miss_test_original += list_val[3]
        miss_original_onnx += list_val[4]
        miss_test_onnx += list_val[5]
        miss_original_coreml += list_val[6]
        miss_test_coreml += list_val[7]
        
        inference_time_original += list_val[9]
        inference_time_onnx += list_val[10]
        inference_time_coreml += list_val[11]
        total_ += np.sum(labels == labels)
        total_datasets += labels.shape[0]
        if i == 1000:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    #print(correct_original, correct_onnx, correct_coreml, total, correct_original*100./total, precision2, recall2, f1)
    #return correct_original*100./total, correct_onnx*100./total, correct_coreml*100./total
    data_writer_run.writerow([model_short_name,framework, training_id, model_name, batch_size, total_,total_datasets, round(accuracy_original*100/total_,2), round(accuracy_onnx*100/total_,2), round(accuracy_coreml*100/total_,2), round(miss_test_original*100/total_datasets,2), round(miss_original_onnx*100/total_datasets,2), round(miss_test_onnx*100/total_datasets,2), round(miss_original_coreml*100/total_datasets,2), round(miss_test_coreml*100/total_datasets,2),'', inference_time_original/total_datasets, inference_time_onnx/total_datasets, inference_time_coreml/total_datasets, '',
                              '{:.0f}m {:.0f}s'.format((inference_time_original/total_datasets) // 60, (inference_time_original/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_onnx/total_datasets) // 60, (inference_time_onnx/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_coreml/total_datasets) // 60, (inference_time_coreml/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)])



In [20]:
#model_name = 'letnet5-keras'
import pandas as pd 
if not os.path.exists(error_path+framework):
    Path(error_path+framework).mkdir(parents=True, exist_ok=True)
#data_file_run = open(error_path+framework+"/{}/runtime_miss-classification_{}.csv".format(model_short_name,model_name), mode='w', newline='',
#                                  encoding='utf-8')
#data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','runtime','original_load_time', 'original_infererence_time', 'runtime_load_time', 
#                          'runtime_inference_time',  'miss_classified_original_runtime_percentage','',  'encoded_miss_classified_original_runtime_percentage','encoded_miss_classified_original_test_runtime_percentage', '', 'accuracy_original', 'accuracy_runtime'])

for round_ in [5,6,7,8,9,10]: #[5,6,7,8,9,10]:
    training_id = round_
    model_short_name = 'Lenet5'
    framework = 'keras'
    path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/Keras/lenet5/'
    since_0 = time.time()
    #model_path = 'tf_Lenet5_mnist_2021-08-24-10:35:35'
    #model_name = 'tf_alexnet_cifar10_2021-08-27-17:05:27'
    model_name = 'tf_Lenet5_mnist_2021-10-28_{}'.format(training_id)
    model = tf.keras.models.load_model(path+ model_name+'.h5')
    t_elapsed_0 = time.time() - since_0
    size0 = os.path.getsize(path+ model_name+'.h5')
    size0
    
    since_1 = time.time()
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    load_time_onnx = time.time() - since_1
    onnx.checker.check_model(onnx_model)
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    
    
    since_1 = time.time()
    coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name))
    load_time_coreml = time.time() - since_1

    flag = 0
    if not os.path.exists(error_path+framework+"/runtime_accuracy_{}2.csv".format(model_short_name)):
        data_file_run2 = open(error_path+framework+"/runtime_accuracy_{}2.csv".format(model_short_name), mode='w', newline='', encoding='utf-8')
    else:
        data_file_run2 = open(error_path+framework+"/runtime_accuracy_{}2.csv".format(model_short_name), mode='a+', newline='', encoding='utf-8')
        flag = 1
    data_writer_run2 = csv.writer(data_file_run2, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    if flag == 0: 
        data_writer_run2.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'test_set','execution_times','accuracy_original', 'accuracy_onnx', 'accuracy_coreml', 'miss_test_original', 'miss_original_onnx', 'miss_test_onnx', 'miss_original_coreml', 'miss_test_coreml','', 'inference_time_original', 'inference_time_onnx', 'inference_time_coreml', '', 'inference_time_original2', 'inference_time_onnx2', 'inference_time_coreml2', 'overral_time'])


    for batch_size in [1]:
        print("################ Batch size: ", batch_size, ' traiining id: ', training_id)
        val_generator =  DataGenerator(x_test, y_test, batch_size=batch_size,  dim = input_shape,  n_classes= nb_classes, to_fit=True, shuffle=True)
        _lets_convert(val_generator, data_writer_run2, batch_size)
        #data_writer_acc.writerow([model_short_name,framework, training_id, model_name, batch_size, correct_original, correct_onnx, correct_coreml])
    data_file_run2.close()
#data_file_acc.close()

################ Batch size:  1  traiining id:  5
0 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.20145702362060547, 0.00039696693420410156, 0.005462169647216797]
50 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.0712440013885498, 0.00028896331787109375, 0.002574920654296875]
100 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.06877803802490234, 0.0005848407745361328, 0.0030341148376464844]
150 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.06469392776489258, 0.0003046989440917969, 0.003962278366088867]
200 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.05756807327270508, 0.0004031658172607422, 0.004266023635864258]
250 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.052102088928222656, 0.0002961158752441406, 0.00442814826965332]
300 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.0569148063659668, 0.0002999305725097656, 0.0038938522338867188]
350 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.07092404365539551, 0.0002658367156982422, 0.0024492740631103516]
400 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.20476698875427246, 0.0005071163177490234, 0.005456209182739258]
450 [10, 10, 10, 0, 0, 0, 0, 0, 

750 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.255511999130249, 0.0004208087921142578, 0.00435185432434082]
800 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.09183478355407715, 0.00035834312438964844, 0.006600856781005859]
850 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.06464624404907227, 0.000308990478515625, 0.002414226531982422]
900 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.12916994094848633, 0.0023469924926757812, 0.004087924957275391]
950 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.07454895973205566, 0.0002720355987548828, 0.0027480125427246094]
1000 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.08181118965148926, 0.0002620220184326172, 0.002454996109008789]
Conversion complete in 1m 35s
################ Batch size:  1  traiining id:  9
0 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.21673583984375, 0.0003418922424316406, 0.002930879592895508]
50 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.09177899360656738, 0.00028705596923828125, 0.002663135528564453]
100 [10, 10, 10, 0, 0, 0, 0, 0, '', 0.23146700859069824, 0.0004258155822753906, 0.006048917770385742]
150 [10,

In [15]:
val_generator =  DataGenerator(x_test, y_test, batch_size=1,  dim = input_shape, 
                               n_classes= nb_classes, 
                               to_fit=True, shuffle=True)
for i, (images, labels) in enumerate(val_generator):
    x, y = images, labels
    break
k_predict = model.predict(x)

In [26]:
y.shape

(1, 10)

In [16]:
onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
onnx.checker.check_model(onnx_model)
size2 = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
#def to_numpy(tensor):
#    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
since_1 = time.time()
ort_inputs = {ort_session.get_inputs()[0].name: x}
ort_outs = ort_session.run(None, ort_inputs)

In [17]:
try:
    np.testing.assert_array_equal(k_predict, ort_outs[0])
except Exception as e:
    print(e)
    for line_ in str(e).split('\n'):
        #print(' ---- : ', line_)
        if 'Mismatched elements' in line_:
            value = line_.replace('Mismatched elements: ', '').strip()
            perc_val = value[value.find("(")+1:value.find(")")]
            print(value, perc_val)
            break


Arrays are not equal

Mismatched elements: 5 / 10 (50%)
Max absolute difference: 1.6805134e-18
Max relative difference: 3.799236e-06
 x: array([[2.118650e-21, 5.607732e-18, 1.213525e-19, 5.153317e-23,
        1.000000e+00, 5.534655e-22, 2.125291e-18, 1.984877e-18,
        2.307871e-17, 8.741971e-13]], dtype=float32)
 y: array([[2.118650e-21, 5.607732e-18, 1.213525e-19, 5.153297e-23,
        1.000000e+00, 5.534634e-22, 2.125290e-18, 1.984877e-18,
        2.307863e-17, 8.741955e-13]], dtype=float32)
5 / 10 (50%) 50%


In [20]:
y0 = to_categorical(np.argmax(k_predict, 1), num_classes = 10)
y1 = to_categorical(np.argmax(ort_outs[0], 1), num_classes = 10)
#y2 = to_categorical(np.argmax(k_predict, 1), num_classes = 10)
try:
    np.testing.assert_array_equal(y, y0)
except Exception as e:
    print(e)
    for line_ in str(e).split('\n'):
        #print(' ---- : ', line_)
        if 'Mismatched elements' in line_:
            value = line_.replace('Mismatched elements: ', '').strip()
            perc_val = value[value.find("(")+1:value.find(")")]
            print(value, perc_val)
            break

In [25]:
y1[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [65]:
ort_outs[0]

array([[7.7533409e-02, 1.7899086e-01, 6.0070749e-02, ..., 7.2314382e-02,
        1.4665432e-01, 7.3706582e-02],
       [7.7533409e-02, 1.7899086e-01, 6.0070749e-02, ..., 7.2314382e-02,
        1.4665432e-01, 7.3706582e-02],
       [7.7533409e-02, 1.7899086e-01, 6.0070749e-02, ..., 7.2314382e-02,
        1.4665432e-01, 7.3706582e-02],
       ...,
       [7.7532411e-02, 1.7899041e-01, 6.0070083e-02, ..., 7.2314888e-02,
        1.4665152e-01, 7.3707536e-02],
       [7.7603646e-02, 1.7887358e-01, 6.0151797e-02, ..., 7.2238758e-02,
        1.4651124e-01, 7.3571399e-02],
       [1.0000000e+00, 3.0505841e-16, 4.4258446e-15, ..., 7.6639033e-16,
        5.0008275e-16, 3.2518154e-17]], dtype=float32)

In [66]:
from keras.utils.np_utils import to_categorical
y2 = to_categorical(np.argmax(ort_outs[0], 1))

In [91]:
x2 = x.flatten()
x2

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [150]:
spec = coreml_model.get_spec()

In [128]:
num_classes = 10
num_anchors = 1
spec.description.output[0].type.multiArrayType.shape.append(num_classes)
spec.description.output[0].type.multiArrayType.shape.append(num_anchors)
#del spec.description.output[1].type.multiArrayType.shape[-1]
#spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(spec)

In [134]:
spec.description

input {
  name: "conv2d_input"
  type {
    multiArrayType {
      shape: 1
      shape: 28
      shape: 28
      shape: 1
      dataType: FLOAT32
      shapeRange {
        sizeRanges {
          lowerBound: 1
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 28
          upperBound: 28
        }
        sizeRanges {
          lowerBound: 28
          upperBound: 28
        }
        sizeRanges {
          lowerBound: 1
          upperBound: 1
        }
      }
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.5.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "5.0"
  }
}

In [93]:
# Print input description to get input shape.
print(coreml_model.get_spec().description.input)

[name: "conv2d_input"
type {
  multiArrayType {
    shape: 1
    shape: 28
    shape: 28
    shape: 1
    dataType: FLOAT32
    shapeRange {
      sizeRanges {
        lowerBound: 1
        upperBound: -1
      }
      sizeRanges {
        lowerBound: 28
        upperBound: 28
      }
      sizeRanges {
        lowerBound: 28
        upperBound: 28
      }
      sizeRanges {
        lowerBound: 1
        upperBound: 1
      }
    }
  }
}
]


In [95]:
print(coreml_model.get_spec().description) 

input {
  name: "conv2d_input"
  type {
    multiArrayType {
      shape: 1
      shape: 28
      shape: 28
      shape: 1
      dataType: FLOAT32
      shapeRange {
        sizeRanges {
          lowerBound: 1
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 28
          upperBound: 28
        }
        sizeRanges {
          lowerBound: 28
          upperBound: 28
        }
        sizeRanges {
          lowerBound: 1
          upperBound: 1
        }
      }
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==2.5.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "5.0"
  }
}



In [158]:
coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}.mlmodel".format(model_short_name, model_name), compute_units=coremltools.ComputeUnit.CPU_ONLY)

output_dict_test = coreml_model.predict({'conv2d_input':x})
####### Mis-classification coreML ######################################

In [151]:
ssd_model = coremltools.models.MLModel(spec)
output_dict_test2 = ssd_model.predict({'conv2d_input':x})

In [153]:
y3_ = to_categorical(np.argmax(output_dict_test2['Identity'], 1))
output_dict_test2['Identity'].shape

(128, 10)

In [160]:
output_dict_test['Identity'].shape

(128, 10)

In [166]:
y3 = to_categorical(np.argmax(output_dict_test['Identity'], 1), num_classes = 10)
y3.shape

(128, 10)

In [167]:
y3

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [165]:
from sklearn.statsmodels.tools import categorical
b = categorical(output_dict_test['Identity'], drop=True)
b.argmax(1)

ModuleNotFoundError: No module named 'sklearn.statsmodels'

In [157]:
k_predict

array([[7.7533409e-02, 1.7899086e-01, 6.0070757e-02, ..., 7.2314389e-02,
        1.4665432e-01, 7.3706582e-02],
       [7.7533409e-02, 1.7899086e-01, 6.0070757e-02, ..., 7.2314389e-02,
        1.4665432e-01, 7.3706582e-02],
       [7.7533409e-02, 1.7899086e-01, 6.0070757e-02, ..., 7.2314389e-02,
        1.4665432e-01, 7.3706582e-02],
       ...,
       [7.7532411e-02, 1.7899041e-01, 6.0070083e-02, ..., 7.2314888e-02,
        1.4665152e-01, 7.3707536e-02],
       [7.7603646e-02, 1.7887357e-01, 6.0151797e-02, ..., 7.2238743e-02,
        1.4651124e-01, 7.3571399e-02],
       [1.0000000e+00, 3.0505955e-16, 4.4258611e-15, ..., 7.6639615e-16,
        5.0008465e-16, 3.2518280e-17]], dtype=float32)

In [101]:
output_dict_test['Identity']

array([[7.7533409e-02, 1.7899086e-01, 6.0070757e-02, ..., 7.2314389e-02,
        1.4665432e-01, 7.3706582e-02],
       [7.7533409e-02, 1.7899086e-01, 6.0070757e-02, ..., 7.2314389e-02,
        1.4665432e-01, 7.3706582e-02],
       [7.7533409e-02, 1.7899086e-01, 6.0070757e-02, ..., 7.2314389e-02,
        1.4665432e-01, 7.3706582e-02],
       ...,
       [7.7532418e-02, 1.7899041e-01, 6.0070086e-02, ..., 7.2314896e-02,
        1.4665154e-01, 7.3707543e-02],
       [7.7603646e-02, 1.7887357e-01, 6.0151801e-02, ..., 7.2238743e-02,
        1.4651124e-01, 7.3571399e-02],
       [1.0000000e+00, 3.0505841e-16, 4.4258446e-15, ..., 7.6639028e-16,
        5.0008465e-16, 3.2518154e-17]], dtype=float32)

In [102]:
y3 = to_categorical(np.argmax(output_dict_test['Identity'], 1))

In [103]:
y3.shape

(128, 5)

In [88]:
for i in range(len(y3)):
    print(y3[i], y[i])

[0. 0. 0. 0. 1.] [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.] [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.] [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.] [0.

In [73]:
np.testing.assert_array_equal(y, y3)

AssertionError: 
Arrays are not equal

(shapes (128, 10), (128, 9) mismatch)
 x: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],...
 y: array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],...

In [71]:
try:
    np.testing.assert_array_equal(y, y3)
except Exception as e:
    #print(e)
    for line_ in str(e).split('\n'):
        #print(' ---- : ', line_)
        if 'Mismatched elements' in line_:
            value = line_.replace('Mismatched elements: ', '').strip()
            perc_val = value[value.find("(")+1:value.find(")")]
            print(value, perc_val)
            break